# <font color="red"> MBA em IA e Big Data</font>
## <span style="color:red">Redes Neurais e Deep Learning</span>
 
### **REDES NEURAIS RECORRENTES - LSTM e GRU**
### Profa. Roseli Ap. Francelin Romero

**PROBLEMA**: SERIE TEMPORAL - Previsão de IPC no periodo 2016.

Este é um problema em que, dados um ano e um mês, a tarefa é prever o IPC (Alimentacao_Vestuario) nos anos de 2016 a 2021. Os dados variam de fev/2016 a set/2021, com 68 observações.






**LSTM PARA REGRESSÃO** A cam. LSTM espera uma matriz como entrada: [samples, time steps, features], onde?

**Samples**: são as observações do dominio da aplicação, normalmente as linhas do dataset.</br> 
**Time steps**: são passos de tempo de separação de uma dada variavel para uma dada observação.</br>
**Features**: são medidas separadas observadas em cada tempo da observação.

 Será apenas a previsão de passo = 1
  

In [1]:

# LSTM for IPC problem with regression framing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow import keras

# converte um array de valores numa matriz de dados 
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)



1 - Fixar a semente de geração e Carregar o dataset do CSV file


In [2]:
# fixar a semente para geração de nos. aleatórios
keras.utils.set_random_seed(7) 
# Carregar o dataset
dataframe = read_csv('IPC.csv', index_col=0)
dataset = dataframe.values
dataset = dataset.astype('float32')
dataframe.head()

AttributeError: module 'keras.api._v2.keras.utils' has no attribute 'set_random_seed'

2 - Como temos dados negativos, normalizar os dados no intervalo [-1,1].

In [ ]:
# normalizar o dataset
scaler = MinMaxScaler(feature_range=(-1, 1))
dataset = scaler.fit_transform(dataset)

3 - Separar os dados em treinamento (67%) e teste (33%)

In [ ]:
# dividir em conjuntos de train and test 
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

6 - Modelar os dados com a LSTM, com timestep = 1, adotando 2 abordagens comparativas:
 - LSTM com 04 neuronios e 100 épocas
 - LSTM com 04 neuronios e 1000 épocas
 

In [ ]:
# reshape em X=t and Y=t+1
#Colocar os dados na forma: X(t) e Y(t+1), chamando a funçao create
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#print('X(t)', 'Y(t+1)')
#for i in range(len(trainX)):
#  print(trainX[i], trainY[i])

# reshape entrada na forma [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
# criando and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)
model.summary()
# fazer as predicoes
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# inverter antes as predicoes
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calcular o RMSE error root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

7 - Fazer o plot da previsão, lembrando que é necessário retornar os dados para a forma original. O que se pode observar?

In [ ]:

# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset), color="blue")
plt.plot(trainPredictPlot, color="green")
plt.plot(testPredictPlot, color="red")
plt.show()

Podemos ver que o modelo LSTM com mais épocas se ajusta melhor nos dados de treino mas nao consegue generalizar bem no conjunto de teste.

8 - Modelar os dados com a GRU, com timestep = 1, adotando 2 abordagens comparativas:
 - LSTM com 04 neuronios e 100 épocas
 - LSTM com 04 neuronios e 1000 épocas

In [ ]:
#Novo codigo para GRU
 

# reshape em X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#print('X(t)', 'Y(t+1)')
#for i in range(len(trainX)):
#  print(trainX[i], trainY[i])
# reshape entrada na forma [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
# criando and fit a rede GRU
model2 = Sequential()
model2.add(GRU(4, input_shape=(1, look_back)))
model2.add(Dense(1))
model2.compile(loss='mean_squared_error', optimizer='adam')
model2.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)
model2.summary()
# fazer as predicoes
trainPredict2 = model2.predict(trainX)
testPredict2 = model2.predict(testX)
# inverter antes as predicoes
trainPredict2 = scaler.inverse_transform(trainPredict2)
trainY2 = scaler.inverse_transform([trainY])
testPredict2 = scaler.inverse_transform(testPredict2)
testY2 = scaler.inverse_transform([testY])
# calcular o RMSE error root mean squared error
trainScore2 = math.sqrt(mean_squared_error(trainY2[0], trainPredict2[:,0]))
print('Train Score: %.2f RMSE' % (trainScore2))
testScore2 = math.sqrt(mean_squared_error(testY2[0], testPredict2[:,0]))
print('Test Score: %.2f RMSE' % (testScore2))

9 - Fazer o plot da previsão da GRU, lembrando que é necessário retornar os dados para a forma original.

In [ ]:

# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset), color="blue")
plt.plot(trainPredictPlot, color="green")
plt.plot(testPredictPlot, color="red")
plt.show()

10- Estabeleça uma comparação entre a LSTM e a GRU.


Resp. Podemos ver que o modelo LSTM com mais épocas se ajusta melhor nos dados de treino mas nao consegue generalizar bem no conjunto de teste. O mesmo acontece com a rede GRU. Observa também que o modelo GRU apresentou  performances similares no conjunto de treinamento que a LSTM, tanto com 100 quanto com 1000 épocas. A série é muito instável e é necessário tentar outras abordagens para obter uma previsão melhor.
